This project uses synthetic generated data from https://infinity.ai/ \
Github https://github.com/toinfinityai/infinity-datasets/tree/main 


#### Parameters
num_reps:  Number of exercise repetitions in the returned time series data.\
watch_location:  Wrist where device is placed (left or right).\
crown_orientation:  Which side the watch crown points (from first-person perspective; left or right).\
ref_xy_rotation:  Rotation (in XY plane) of the reference orientation in radians (simulates the xArbitraryZVertical reference in the Apple CoreMotion SDK).\
rel_baseline_speed:  Baseline speed of animation, relative to default (natural) speed.\
max_rel_speed_change:  Maximum speed change introduced, relative to baseline speed.\
trim_start_frac:  Fraction of seed animation (from start to midpoint) to truncate at the start.\
trim_end_frac:  Fraction of seed animation (from start to midpoint) to truncate at the end.\
kinematic_noise_factor:  Scaling factor used to adjust the amount of kinematic noise added in the simulated movement.\
wrist_offset_deg:  Fixed rotation offset applied to the supination/pronation axis of the wrists, in degrees. Negative values correspond to supination.\
randomize_body_shape:  If True, the avatar's body shape is randomized.\
frames_per_second:  Sampling rate of exported time series and video.\
image_width:  Width dimension of the rendered video, in pixels.\
image_height:  Height dimension of the rendered video, in pixels.


In [5]:
import pandas as pd
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None) 
#pd.reset_option('display.max_rows')
#pd.reset_option('display.max_columns')

In [66]:
#function for retrieving multiple csv/json files

import glob
import pandas as pd
import json
from pandas import json_normalize
import os
import numpy as np

def create_combined_dataframe(directory):
    """
    Create a combined DataFrame for each set of CSV and JSON files in the specified directory.

    Parameters:
    - directory (str): The directory containing CSV and JSON files.

    Returns:
    - List[pd.DataFrame]: A list of DataFrames where each DataFrame combines CSV and JSON data, with each parameter in the JSON file having its own column.
    """
    combined_dfs = []
    
    # creates list of file names matching the pattern '*.csv' in the specified directory
    csv_files = glob.glob(f'{directory}/*.csv')

    # Process each CSV file
    for csv_file in csv_files:
        csv_df = pd.read_csv(csv_file, index_col=0)

        # json file path
        json_file = os.path.splitext(csv_file)[0] + '_params.json'

        # Read and normalize json file, create df
        with open(json_file, 'r') as f:
            json_data = json.load(f)

        json_df = json_normalize(json_data)
        json_df.columns = [col.replace('params.', '') for col in json_df.columns]
        json_df = pd.concat([json_df] * len(csv_df), ignore_index=True)

        #combined data frames
        combined_df = pd.concat([json_df, csv_df], axis=1)
        combined_dfs.append(combined_df)

    return combined_dfs

In [67]:
#creating list of armraise dataframes
armraise_path = '../data/basic_motions/armraise'
armraise_dataframes = create_combined_dataframe(armraise_path)

In [68]:
armraise_dataframes[0]

,exercise,num_reps,watch_location,crown_orientation,ref_xy_rotation,rel_baseline_speed,max_rel_speed_change,trim_start_frac,trim_end_frac,kinematic_noise_factor,wrist_offset_deg,use_random_motion,randomize_body_shape,frames_per_second,image_width,image_height,random_seed,seconds_per_rep,num_random_frames,rep_count_from_intermediate,rep_count_from_start,ref_xy_rotation,time,rotation_matrix_m11,rotation_matrix_m12,rotation_matrix_m13,rotation_matrix_m21,rotation_matrix_m22,rotation_matrix_m23,rotation_matrix_m31,rotation_matrix_m32,rotation_matrix_m33
0,ARM_RAISE-DUMBBELL,12,RIGHT,LEFT,4.818261,0.851037,0.024017,0.045082,0.057994,0.980721,9.201472,False,True,20,480,480,677412957,1.0,100,0.473684,0.000000,4.818261,0.00,0.313452,-0.218860,-0.924039,0.097586,0.975350,-0.197910,0.944576,-0.028138,0.327084
1,ARM_RAISE-DUMBBELL,12,RIGHT,LEFT,4.818261,0.851037,0.024017,0.045082,0.057994,0.980721,9.201472,False,True,20,480,480,677412957,1.0,100,0.490279,0.016595,4.818261,0.05,0.319075,-0.216120,-0.922758,0.092488,0.976105,-0.196634,0.943206,-0.022603,0.331439
2,ARM_RAISE-DUMBBELL,12,RIGHT,LEFT,4.818261,0.851037,0.024017,0.045082,0.057994,0.980721,9.201472,False,True,20,480,480,677412957,1.0,100,0.506874,0.033190,4.818261,0.10,0.331619,-0.214452,-0.918716,0.085394,0.976647,-0.197151,0.939541,-0.013074,0.342188
3,ARM_RAISE-DUMBBELL,12,RIGHT,LEFT,4.818261,0.851037,0.024017,0.045082,0.057994,0.980721,9.201472,False,True,20,480,480,677412957,1.0,100,0.523469,0.049785,4.818261,0.15,0.350453,-0.213280,-0.911973,0.077130,0.976991,-0.198847,0.933399,-0.000654,0.358839
4,ARM_RAISE-DUMBBELL,12,RIGHT,LEFT,4.818261,0.851037,0.024017,0.045082,0.057994,0.980721,9.201472,False,True,20,480,480,677412957,1.0,100,0.540063,0.066379,4.818261,0.20,0.379883,-0.213341,-0.900097,0.067529,0.976841,-0.203030,0.922566,0.016345,0.385492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
709,ARM_RAISE-DUMBBELL,12,RIGHT,LEFT,4.818261,0.851037,0.024017,0.045082,0.057994,0.980721,9.201472,False,True,20,480,480,677412957,1.0,100,12.370983,11.897299,4.818261,35.45,0.515902,-0.055648,-0.854838,-0.001645,0.997822,-0.065948,0.856646,0.035429,0.514687
710,ARM_RAISE-DUMBBELL,12,RIGHT,LEFT,4.818261,0.851037,0.024017,0.045082,0.057994,0.980721,9.201472,False,True,20,480,480,677412957,1.0,100,12.387749,11.914065,4.818261,35.50,0.458979,-0.059240,-0.886470,0.014939,0.998148,-0.058968,0.888321,0.013822,0.459014
711,ARM_RAISE-DUMBBELL,12,RIGHT,LEFT,4.818261,0.851037,0.024017,0.045082,0.057994,0.980721,9.201472,False,True,20,480,480,677412957,1.0,100,12.404509,11.930825,4.818261,35.55,0.416546,-0.063375,-0.906903,0.030470,0.997980,-0.055744,0.908604,-0.004413,0.417636
712,ARM_RAISE-DUMBBELL,12,RIGHT,LEFT,4.818261,0.851037,0.024017,0.045082,0.057994,0.980721,9.201472,False,True,20,480,480,677412957,1.0,100,12.421261,11.947576,4.818261,35.60,0.383611,-0.067923,-0.920994,0.048168,0.997406,-0.053496,0.922238,-0.023840,0.385887


In [69]:
armraise_dataframes[0]['watch_location'].unique()

array(['RIGHT'], dtype=object)

In [70]:
armraise_dataframes[0]['crown_orientation'].unique()

array(['LEFT'], dtype=object)

Learning about changing rotational transformation matrix to roll-pitch-yaw

There are many csv's to import, but I am just looking at one exercise right now while I learn about converting a rotational matrix to roll-pitch-yaw. I will import the rest afterwards.

https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.transform.Rotation.as_euler.html \
https://eigen.tuxfamily.org/dox/group__Geometry__Module.html#title40 \
https://github.com/zivid/zivid-python-samples/blob/master/source/applications/advanced/hand_eye_calibration/pose_conversions.py 

In [75]:
import pandas as pd
from scipy.spatial.transform import Rotation as R

def add_euler_columns(df):
    rotation_matrix_columns = [
        'rotation_matrix_m11', 'rotation_matrix_m12', 'rotation_matrix_m13',
        'rotation_matrix_m21', 'rotation_matrix_m22', 'rotation_matrix_m23',
        'rotation_matrix_m31', 'rotation_matrix_m32', 'rotation_matrix_m33'
    ]

    rotation_matrices = df[rotation_matrix_columns].values.reshape(-1, 3, 3)
    euler_angles = R.from_matrix(rotation_matrices).as_euler("xyz")

    euler_df = pd.DataFrame(euler_angles, columns=['euler_x', 'euler_y', 'euler_z'])
    
    
    df = df.drop(rotation_matrix_columns, axis=1)
    df = pd.concat([df, euler_df], axis=1)

    return df

In [76]:
# Update each DataFrame in the list with euler angles instead of rotation matrix
for i in range(len(armraise_dataframes)):
    armraise_dataframes[i] = add_euler_columns(armraise_dataframes[i])


In [77]:
armraise_dataframes[50]

,exercise,num_reps,watch_location,crown_orientation,ref_xy_rotation,rel_baseline_speed,max_rel_speed_change,trim_start_frac,trim_end_frac,kinematic_noise_factor,wrist_offset_deg,use_random_motion,randomize_body_shape,frames_per_second,image_width,image_height,random_seed,seconds_per_rep,num_random_frames,rep_count_from_intermediate,rep_count_from_start,ref_xy_rotation,time,euler_x,euler_y,euler_z
0,ARM_RAISE-DUMBBELL,10,LEFT,RIGHT,0.879822,1.059816,0.032639,0.103032,0.156034,0.826002,-11.826704,False,True,20,480,480,1984524181,1.0,100,0.466667,0.000000,0.879822,0.00,-1.128989,0.313235,1.891659
1,ARM_RAISE-DUMBBELL,10,LEFT,RIGHT,0.879822,1.059816,0.032639,0.103032,0.156034,0.826002,-11.826704,False,True,20,480,480,1984524181,1.0,100,0.492401,0.025734,0.879822,0.05,-1.060150,0.327630,1.943922
2,ARM_RAISE-DUMBBELL,10,LEFT,RIGHT,0.879822,1.059816,0.032639,0.103032,0.156034,0.826002,-11.826704,False,True,20,480,480,1984524181,1.0,100,0.518163,0.051496,0.879822,0.10,-0.968391,0.333875,2.012972
3,ARM_RAISE-DUMBBELL,10,LEFT,RIGHT,0.879822,1.059816,0.032639,0.103032,0.156034,0.826002,-11.826704,False,True,20,480,480,1984524181,1.0,100,0.543954,0.077288,0.879822,0.15,-0.885528,0.322451,2.076853
4,ARM_RAISE-DUMBBELL,10,LEFT,RIGHT,0.879822,1.059816,0.032639,0.103032,0.156034,0.826002,-11.826704,False,True,20,480,480,1984524181,1.0,100,0.569774,0.103108,0.879822,0.20,-0.782802,0.293343,2.155573
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,ARM_RAISE-DUMBBELL,10,LEFT,RIGHT,0.879822,1.059816,0.032639,0.103032,0.156034,0.826002,-11.826704,False,True,20,480,480,1984524181,1.0,100,10.307658,9.840991,0.879822,18.55,-0.499118,0.155347,2.207591
372,ARM_RAISE-DUMBBELL,10,LEFT,RIGHT,0.879822,1.059816,0.032639,0.103032,0.156034,0.826002,-11.826704,False,True,20,480,480,1984524181,1.0,100,10.333505,9.866838,0.879822,18.60,-0.620559,0.225231,2.133814
373,ARM_RAISE-DUMBBELL,10,LEFT,RIGHT,0.879822,1.059816,0.032639,0.103032,0.156034,0.826002,-11.826704,False,True,20,480,480,1984524181,1.0,100,10.359340,9.892673,0.879822,18.65,-0.744739,0.279013,2.056999
374,ARM_RAISE-DUMBBELL,10,LEFT,RIGHT,0.879822,1.059816,0.032639,0.103032,0.156034,0.826002,-11.826704,False,True,20,480,480,1984524181,1.0,100,10.385163,9.918496,0.879822,18.70,-0.839479,0.310311,1.999223
